In [4]:
# %pwd


In [5]:
# cd "C:\Users\Om gunjal\OneDrive\Documents\vs\MedAI\MedAi"

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [8]:

extracted_data=load_pdf_file(data='Data/')

In [9]:

#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:

text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5859


In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

embeddings = download_hugging_face_embeddings()


C:\Users\Om gunjal\AppData\Local\Temp\ipykernel_11392\1787422007.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
c:\Users\Om gunjal\.conda\envs\Medaibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [13]:
# ✅ Embeddings
# ✅ Embeddings (keep as is)
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ✅ Vector DB: FAISS
from langchain.vectorstores import FAISS

# Example: You must define `text_chunks` first. Example placeholder:
# from langchain_community.document_loaders import PyPDFLoader
# loader = PyPDFLoader("yourfile.pdf")
# documents = loader.load()
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# text_chunks = splitter.split_documents(documents)

# Create vectorstore
vectorstore = FAISS.from_documents(documents=text_chunks, embedding=embeddings)

# Use as retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke("What is Acne?")
print(retrieved_docs)

# ✅ Local LLM using Hugging Face (use flan-t5-small instead of Falcon)
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "google/flan-t5-small"  # ✅ Change here

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to("cpu")

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=500)
llm = HuggingFacePipeline(pipeline=pipe)

# Example usage
question = "What is Acne?"
context = "\n".join([doc.page_content for doc in retrieved_docs])

prompt = f"Answer the following question using the context below:\n\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"

response = llm(prompt)
print(response)



[Document(id='33d7a780-9fe2-4586-a4c8-e742929f9f4a', metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637, 'page': 39, 'page_label': '40'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'), Document(id='687ee542-f7e2-447d-9c16-bcd376817541', metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637, 'page': 38, 'page_label': '39'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to

c:\Users\Om gunjal\.conda\envs\Medaibot\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Om gunjal\.cache\huggingface\hub\models--google--flan-t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular

skin disorder in which the sebaceous glands become inflamed


In [ ]:
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.document_loaders import PyPDFLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline

from src.prompt import system_prompt

# --------------------------
# 📄 1️⃣ PDF file to load
# --------------------------
# 🔥 CHANGE THIS to your PDF path if needed
loader = PyPDFLoader("Data/Medical_book.pdf")
pages = loader.load()

# --------------------------
# ✂️ 2️⃣ Text chunk settings
# --------------------------
# You can change chunk_size and chunk_overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_chunks = text_splitter.split_documents(pages)

# --------------------------
# 💬 3️⃣ Embedding model
# --------------------------
# You can change model_name here if needed
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# --------------------------
# 💾 4️⃣ FAISS index folder
# --------------------------
# You can change folder name here if you like
index_folder = "faiss_index"

if os.path.exists(index_folder):
    print("🔁 Loading existing FAISS index...")
    vectorstore = FAISS.load_local(index_folder, embeddings, allow_dangerous_deserialization=True)
else:
    print("💥 Creating new FAISS index...")
    vectorstore = FAISS.from_documents(documents=text_chunks, embedding=embeddings)
    vectorstore.save_local(index_folder)
    print("✅ Index saved locally.")


# --------------------------
# 🔍 5️⃣ Create retriever
# --------------------------
# You can change k (how many similar docs to return)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# --------------------------
# 🤖 6️⃣ LLM model
# --------------------------
# CHANGE model_name to any local model you want (e.g., "google/flan-t5-base")
model_name = "google/flan-t5-small"   #model_name = "google/flan-t5-base"  # ✅ Change here for fast OP


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to("cpu")  # or "cuda" if using GPU

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=500, temperature=0.3)
llm = HuggingFacePipeline(pipeline=pipe)

# --------------------------
# 💬 7️⃣ Example question
# --------------------------
# CHANGE question text below
question = "What is Acne?"

retrieved_docs = retriever.invoke(question)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
context = context[:1500]



prompt = (
    system_prompt
    + "\n\n"
    + f"{context}\n\nQuestion: {question}\nAnswer:"
)


response = llm.invoke(prompt)
print("🩺 Answer:")
print(response)


🔁 Loading existing FAISS index...


Device set to use cpu


🩺 Answer:
a skin disorder in which the sebaceous glands become inflamed
